In [7]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col,when


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

In [2]:
Products = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Products") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

Discounts = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Discounts") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [18]:


Products\
    .join(Discounts,Products.category == Discounts.category,"left")\
    .withColumn("final_price",when((col("price")-(col("discount")/100*col("price"))).isNull(),Products.price).otherwise((col("price")-(col("discount")/100*col("price")))))\
    .select("product_id","final_price",Products.category)\
    .show()

# price - (discount/100)*price)

+----------+-----------+-----------+
|product_id|final_price|   category|
+----------+-----------+-----------+
|         4|      500.0|       Home|
|         1|      900.0|Electronics|
|         3|     1080.0|Electronics|
|         2|       40.0|   Clothing|
+----------+-----------+-----------+

